Выполнить анализ датасета - полно и всесторонне изучить данные, предположить, какую задачу может потребоваться решить на основе этих данных, построить при необходимости таблицы, диаграммы, графики, определить зависимости. 
Предположить и обосновать, какие ML алгоритмы или модели можно использовать для описания представленных данных.

In [ ]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_task_hcl.csv to test_task_hcl.csv


In [ ]:
df=pd.read_csv("test_task_hcl.csv")
df.head(10)

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,HCLTECH,EQ,1-Jan-16,855.10,855.10,855.10,843.50,845.6,845.85,846.83,485196,410880355.7,9066,291168,60.01
1,HCLTECH,EQ,4-Jan-16,845.85,841.00,854.95,837.45,846.95,845.95,843.36,1086036,915919738.5,43114,540113,49.73
2,HCLTECH,EQ,5-Jan-16,845.95,848.95,851.05,836.40,844,842.80,842.45,732691,617257642.2,41363,456084,62.25
3,HCLTECH,EQ,6-Jan-16,842.80,853.70,853.70,837.00,841,841.40,843.19,1398935,1179563879,58050,971039,69.41
4,HCLTECH,EQ,7-Jan-16,841.40,836.10,837.90,819.00,825.5,825.05,827.47,1450657,1200371675,51218,951629,65.6
5,HCLTECH,EQ,8-Jan-16,825.05,828.00,836.75,813.80,831.05,828.35,824.68,1688435,1392420250,46806,1163945,68.94
6,HCLTECH,EQ,11-Jan-16,828.35,815.00,833.00,811.10,815,814.55,816.61,1267638,1035171029,50618,770967,60.82
7,HCLTECH,EQ,12-Jan-16,814.55,818.00,819.00,798.05,809.65,810.05,805.53,1157664,932537991.3,54962,683855,59.07
8,HCLTECH,EQ,13-Jan-16,810.05,814.80,829.85,804.30,829,826.40,814.67,1579234,1286553554,54897,1017038,64.4
9,HCLTECH,EQ,14-Jan-16,826.40,810.05,841.80,810.05,830.05,828.85,832.74,2540610,2115655504,56588,1688083,66.44


# Анализ, чистка и подготовка данных

In [ ]:
# проверка на наличие дубликатов строк. результат - False по всем 1326 строкам, т.е. дублей строк нет.
df.duplicated().value_counts()

In [ ]:
# проверка отсутствующих значений. результат - Nan отсутствуют.
df.isnull().sum()

In [ ]:
# количество значений 0. результат - 0 отсутствуют.
(df == 0).sum()

In [ ]:
df.nunique()

In [ ]:
df.dtypes

In [ ]:
# анализ и преобразование дат obj -> datetime64
date_lengths=df.Date.str.len()
date_lengths.value_counts()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date']

In [ ]:
# проверка данных столбца Symbol
df['Symbol'].value_counts()

In [ ]:
# проверка данных столбца Series
series_of_stock=df['Series'].value_counts()
series_of_stock

In [ ]:
df[(df['Series'] == 'BL') | (df['Series'] == 'BT')]

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
157,HCLTECH,BT,2016-08-22,580.0,783.65,784.05,783.65,-,784.05,783.86,99,77602.55,2,-,-
799,HCLTECH,BL,2019-03-25,1420.0,1024.85,1024.85,1024.85,1024.85,1024.85,1024.85,186783,191424557.6,1,186783,100


In [ ]:
df.duplicated('Date').value_counts()

За весь анализируемый период было 2 случая торговли особых серий акций (BT, BL) c иными от серии EQ характеристиками (право собственности, право голоса, дивиденды, номинальная стоимость и т.д.).

 Для чистоты дальнейший анализ будет проводится без этих данных.

In [ ]:
# сохраним акции series BT и BL в df_BT_BL, возможно они будут полезны аналитику при анализе.
df_BT_BL = df[(df.Series == 'BT') | (df.Series == 'BL')]
df_BT_BL

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
157,HCLTECH,BT,2016-08-22,580.0,783.65,784.05,783.65,-,784.05,783.86,99,77602.55,2,-,-
799,HCLTECH,BL,2019-03-25,1420.0,1024.85,1024.85,1024.85,1024.85,1024.85,1024.85,186783,191424557.6,1,186783,100


In [ ]:
# создадим df_EQ для последующей работы
df_EQ = df[df.Series == 'EQ'].reset_index(drop=True)
df_EQ

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,HCLTECH,EQ,2016-01-01,855.10,855.10,855.10,843.50,845.6,845.85,846.83,485196,410880355.7,9066,291168,60.01
1,HCLTECH,EQ,2016-01-04,845.85,841.00,854.95,837.45,846.95,845.95,843.36,1086036,915919738.5,43114,540113,49.73
2,HCLTECH,EQ,2016-01-05,845.95,848.95,851.05,836.40,844,842.80,842.45,732691,617257642.2,41363,456084,62.25
3,HCLTECH,EQ,2016-01-06,842.80,853.70,853.70,837.00,841,841.40,843.19,1398935,1179563879,58050,971039,69.41
4,HCLTECH,EQ,2016-01-07,841.40,836.10,837.90,819.00,825.5,825.05,827.47,1450657,1200371675,51218,951629,65.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,HCLTECH,EQ,2021-05-04,907.70,910.00,915.00,899.25,902.95,900.95,907.88,6217026,5644301124,168209,4062318,65.34
1320,HCLTECH,EQ,2021-05-05,900.95,903.95,910.80,900.60,910.15,909.40,906.28,2696835,2444085189,101282,1203503,44.63
1321,HCLTECH,EQ,2021-05-06,909.40,912.00,923.90,901.25,915.2,916.00,911.90,6319104,5762420236,143748,3197706,50.6
1322,HCLTECH,EQ,2021-05-07,916.00,918.00,921.40,913.70,915.55,916.40,917.07,"36,51,558","3,34,87,26,455.40",78952,"21,14,655",57.91


In [ ]:
# ф-я, которая убирает "," из значений и преобразует их в float
def multy_float(column):
  float_col = [float(row.replace(',','')) for row in column]
  return np.array(float_col)

In [ ]:
# чистка + преобразование obj -> float
df_EQ.loc[:, 'Last Price'] = multy_float(df_EQ['Last Price'])
df_EQ.loc[:, 'Total Traded Quantity'] = multy_float(df_EQ['Total Traded Quantity'])
df_EQ.loc[:, 'Turnover'] = multy_float(df_EQ['Turnover'])
df_EQ.loc[:, 'No. of Trades'] = multy_float(df_EQ['No. of Trades'])
df_EQ.loc[:, 'Deliverable Qty'] = multy_float(df_EQ['Deliverable Qty'])
df_EQ.loc[:, '% Dly Qt to Traded Qty'] = multy_float(df_EQ['% Dly Qt to Traded Qty'])

In [ ]:
# проверка отработки функции
df_EQ.dtypes

In [ ]:
# создадим столбец с новым показателем, который отображает ежедневное изменение стоимости акций '% of Price change'
df_EQ['% of Price change'] = ""
df_EQ['% of Price change'][0] = 0
for i in range(len(df_EQ)):
  df_EQ['% of Price change'][i] = round((df_EQ['Close Price'][i]/df_EQ['Prev Close'][i]-1)*100, 3)
df_EQ['% of Price change'] = df_EQ['% of Price change'].astype('float',copy=False)

In [ ]:
# создадим столбец с новым показателем, который отображает абсолютное изменение стоимости акций за день 'absolut day Price change'
df_EQ['absolut day Price change'] = ""
for i in range(len(df_EQ)):
  df_EQ['absolut day Price change'][i] = df_EQ['Close Price'][i] - df_EQ['Prev Close'][i]
df_EQ['absolut day Price change'] = df_EQ['absolut day Price change'].astype('float',copy=False)

In [ ]:
# построение корреляционной матрицы
corr = df_EQ.corr()
corr.style.background_gradient(cmap='PuBu')

,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty,% of Price change,absolut day Price change
Prev Close,1.000000,0.994501,0.992900,0.992730,0.991182,0.991192,0.992517,-0.235446,-0.016398,-0.031986,-0.302973,0.004868,-0.073043,-0.066161
Open Price,0.994501,1.000000,0.998200,0.998051,0.996408,0.996411,0.997778,-0.228735,-0.008169,-0.025485,-0.300115,0.002553,-0.001215,0.014595
High Price,0.992900,0.998200,1.000000,0.997753,0.998438,0.998469,0.999234,-0.211766,0.008219,-0.010927,-0.288548,-0.012573,0.029478,0.042164
Low Price,0.992730,0.998051,0.997753,1.000000,0.998410,0.998497,0.999179,-0.242750,-0.022349,-0.042833,-0.313092,0.011084,0.030239,0.043650
Last Price,0.991182,0.996408,0.998438,0.998410,1.000000,0.999897,0.999372,-0.229160,-0.009276,-0.029667,-0.303432,-0.001971,0.054866,0.065866
Close Price,0.991192,0.996411,0.998469,0.998497,0.999897,1.000000,0.999458,-0.230043,-0.010126,-0.030801,-0.303808,-0.000770,0.055686,0.066566
Average Price,0.992517,0.997778,0.999234,0.999179,0.999372,0.999458,1.000000,-0.226950,-0.006720,-0.027016,-0.301667,-0.002585,0.039995,0.052498
Total Traded Quantity,-0.235446,-0.228735,-0.211766,-0.242750,-0.229160,-0.230043,-0.226950,1.000000,0.961212,0.868759,0.793716,-0.495166,0.059419,0.040661
Turnover,-0.016398,-0.008169,0.008219,-0.022349,-0.009276,-0.010126,-0.006720,0.961212,1.000000,0.889799,0.735070,-0.501100,0.062705,0.047253
No. of Trades,-0.031986,-0.025485,-0.010927,-0.042833,-0.029667,-0.030801,-0.027016,0.868759,0.889799,1.000000,0.713227,-0.431467,0.019524,0.008921


Чем выше значение, тем сильнее взаимосвязь между данными. Отрицательные значения означают обратную взаимоствязь.

Из результатов корреляционной матрицы можно сделать вывод о сильной прямой связи цен между собой и сильной прямой связи количестенных показателей (торгуемое кол-во, оборот, кол-во сделок )

#Интерактивные* графики для инвестиционных аналитиков 


**отображение информации при наведении курсора, возможность приближения, включения/выключения показателей и т.д.*


In [ ]:
# "японские свечи" (динамика стоимости акций)
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df_EQ['Date'],
                open=df_EQ['Open Price'],
                high=df_EQ['High Price'],
                low=df_EQ['Low Price'],
                close=df_EQ['Close Price'])])
fig.update_layout(xaxis_rangeslider_visible=False, title='NSE: HCLTECH (HCL Technologies)', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [ ]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
pip install plotly

In [ ]:
pip install cufflinks

In [ ]:
pip install cufflinks --upgrade

In [ ]:
pip install chart_studio

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
# График дневного изменения цен на акции

configure_plotly_browser_state()
df2 = df_EQ[['% of Price change', 'absolut day Price change', 'Date']].set_index('Date')
fig2 = df2.iplot(kind='bar', 
                 title='Day change price', 
                 xTitle='Date', 
                 yTitle='level of change')

In [ ]:
configure_plotly_browser_state()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['% of Price change'], name='% of Price change'))
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['absolut day Price change'], name='absolut day Price change'))

fig3.show()

In [ ]:
# графики изменения дневных показателей

configure_plotly_browser_state()
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['Total Traded Quantity'], name='Total Traded Quantity'))
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['No. of Trades'], name='No. of Trades'))
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['Deliverable Qty'], name='Deliverable Qty'))
fig3.add_trace(go.Scatter(x=df_EQ['Date'], y=df_EQ['% Dly Qt to Traded Qty'], name='% Dly Qt to Traded Qty'))

fig3.show()

# Построение и оценка моделей
Создадим и обучим модель, делающую прогноз 'High Price'.
Для нашей задачи подойдут регрессионые методы. 

In [ ]:
# импорт библиотек
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# введем переменные для создания модели и получения прогноза: X - features, Y - target
X = df_EQ[['Open Price', 'Low Price', 'Close Price', 'Average Price', 'Total Traded Quantity', 'No. of Trades', 'Deliverable Qty', '% of Price change',]]
Y = df_EQ['High Price']

In [ ]:
# разделим набор данных на обучающий и проверочный
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [ ]:
TestModels = pd.DataFrame()
tmp = {}

classifiers = [
    svm.SVR(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression(),
    ]

for item in classifiers:
    my_model = item
    my_model.fit(X_train, Y_train)
    predictions = my_model.predict(X_valid)
    #print(my_model.predict(X_valid),'\n')
    tmp['MAE'] = round(mean_absolute_error(predictions, Y_valid),3)
    tmp['MSE'] = round(mean_squared_error(predictions, Y_valid),3)
    TestModels = TestModels.append([tmp])

TestModels.index = ['SVR', 'BayesianRidge', 'LassoLars', 'ARDRegression', 'PassiveAggressiveRegressor', 'TheilSenRegressor', 'LinearRegression']
TestModels

,MAE,MSE
SVR,121.896,24925.097
BayesianRidge,2.908,17.258
LassoLars,25.481,1095.864
ARDRegression,2.907,18.184
PassiveAggressiveRegressor,473.902,664307.426
TheilSenRegressor,3.047,18.902
LinearRegression,2.909,17.264


Наилучший прогноз (минимальное значение средней абсолютной и среднеквадратичной ошибки) дают модели, обученые методом ARDRegression и BayesianRidge.



Аналогичным образом можно обучить модель для прогнозирования значений других переменных акций HCLTech